In [6]:
from sklearn.metrics import f1_score, balanced_accuracy_score, mean_squared_error, confusion_matrix, recall_score, precision_score, accuracy_score, log_loss
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments, BertweetTokenizer, BertConfig
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
import torch
import json
import os
import re
import gc

import pandas as pd
import os
import numpy as np
from sklearn.manifold import TSNE
import torch
from sentence_transformers import SentenceTransformer, util

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

In [1]:
path = './Webis17HeadlineClsData/'  # Webis17HeadlineClsData or Webis17HeadlineArticleClsData
root = "./"

In [3]:
df_train = pd.read_csv(path+'train.csv')
df_validation = pd.read_csv(path+'validation.csv')
df_test = pd.read_csv(path+'test.csv')

In [4]:
df_train.head()

,postText,truthClass
0,UK’s response to modern slavery leaving victim...,no-clickbait
1,this is good,clickbait
2,"The ""forgotten"" Trump roast: Relive his brutal...",no-clickbait
3,Meet the happiest #dog in the world!,clickbait
4,Tokyo's subway is shut down amid fears over an...,no-clickbait


In [5]:
len(df_train), len(df_validation), len(df_test)

(19538, 2459, 18979)

In [12]:
df_train.dtypes

postText      object
truthClass    object
dtype: object

In [13]:
df_train['postText'] = df_train['postText'].astype(str)
df_validation['postText'] = df_validation['postText'].astype(str)
df_test['postText'] = df_test['postText'].astype(str)

In [14]:
len(df_train), len(df_validation), len(df_test)

(19538, 2459, 18979)

In [15]:
postText_train = df_train['postText'].tolist()
# generatedPostText_train_cb = train_cb['generatedPostText'].tolist()

postText_val = df_validation['postText'].tolist()
# generatedPostText_train_ncb = train_ncb['generatedPostText'].tolist()

postText_test = df_test['postText'].tolist()
# generatedPostText_test_cb = test_cb['generatedPostText'].tolist()


In [16]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [17]:
train_emb = model.encode(postText_train)
val_emb = model.encode(postText_val)
test_emb = model.encode(postText_test)

In [23]:
train_emb.shape, val_emb.shape, test_emb.shape

((19538, 384), (2459, 384), (18979, 384))

In [19]:
X = list(train_emb)

X_embedded = TSNE(n_components=2).fit_transform(X)

/home/charmichokshi4444/venv/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/charmichokshi4444/venv/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [21]:
X1 = list(val_emb)

X1_embedded = TSNE(n_components=2).fit_transform(X1)

/home/charmichokshi4444/venv/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/home/charmichokshi4444/venv/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [59]:
X2 = list(test_emb)

X2_embedded = TSNE(n_components=2).fit_transform(X2)

/home/charmichokshi4444/venv/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/charmichokshi4444/venv/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [60]:
X2_embedded.shape

(18979, 2)

In [22]:
X1_embedded.shape

(2459, 2)

In [26]:
import pandas as pd
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings.shape

(19538, 2)

In [29]:
df_embeddings1 = pd.DataFrame(X1_embedded)
df_embeddings1.shape

(2459, 2)

In [61]:
df_embeddings2 = pd.DataFrame(X2_embedded)
df_embeddings2.shape

(18979, 2)

In [71]:
myx = pd.concat([df_embeddings, df_embeddings2, df_embeddings1])
myx.shape

(40976, 2)

In [72]:
# labels=[]
lab = ['Train']*19538
lab1 = ['Validation']*2459
lab2 = ['Test']*18979
print(len(lab))
# lables = lab
lab.extend(lab2)
lab.extend(lab1)

19538


In [73]:
len(lab)

40976

In [74]:

myx = myx.rename(columns={0:'x',1:'y'})
myx = myx.assign(label=lab)

In [75]:
myx.head()

,x,y,label
0,-27.041845,-0.651166,Train
1,50.885303,-50.484047,Train
2,-16.528938,31.860542,Train
3,14.011378,3.859091,Train
4,-66.635811,-20.912169,Train


In [77]:
import plotly.express as px

fig = px.scatter(
    myx, x='x', y='y',
    color='label', labels={'color': 'label'},
    title = 'TSNE plot of Traiin, Test, Validation Data')
fig.show()